In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

In [33]:
bureau = pd.read_csv('bureau.csv')

In [3]:
train = pd.read_csv('application_train.csv')

In [4]:
bureau.head()
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count() \
        .rename(columns = { 'SK_ID_BUREAU': 'previous_loan_count'})

In [5]:
train = train.merge(previous_loan_counts, on='SK_ID_CURR', how='left')
train['previous_loan_count'] = train['previous_loan_count'].fillna(0)

In [6]:
def kde_target(df, var_name):
    
    corr = df['TARGET'].corr(df[var_name])
    avg_repaid = df.ix[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize=(12, 6))
    sns.kdeplot(df.ix[df['TARGET'] == 0, var_name], label='Target == 0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label='Target == 1')
    
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution'%var_name); plt.legend()
    
    print('The correlation between %s and TARGET is %s'%(var_name, str(corr)))
    print('Median value for loan that was not repaid is %s'%(str(avg_not_repaid)))
    print('Median value for loan that was repaid is %s'%(str(avg_repaid)))

In [7]:
kde_target(train, 'EXT_SOURCE_3')

The correlation between EXT_SOURCE_3 and TARGET is -0.178918697628
Median value for loan that was not repaid is 0.3791004854
Median value for loan that was repaid is 0.546023197005


In [8]:
kde_target(train, 'previous_loan_count')

The correlation between previous_loan_count and TARGET is -0.0100197156707
Median value for loan that was not repaid is 3.0
Median value for loan that was repaid is 4.0


In [9]:
bureau_agg = bureau.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index=False) \
                .agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()

SK_ID_CURR DAYS_CREDIT                               CREDIT_DAY_OVERDUE  \
                   count         mean  max   min   sum              count   
0     100001           7  -735.000000  -49 -1572 -5145                  7   
1     100002           8  -874.000000 -103 -1437 -6992                  8   
2     100003           4 -1400.750000 -606 -2586 -5603                  4   
3     100004           2  -867.000000 -408 -1326 -1734                  2   
4     100005           3  -190.666667  -62  -373  -572                  3   

                 ...    DAYS_CREDIT_UPDATE                               \
  mean max min   ...                 count        mean  max   min   sum   
0  0.0   0   0   ...                     7  -93.142857   -6  -155  -652   
1  0.0   0   0   ...                     8 -499.875000   -7 -1185 -3999   
2  0.0   0   0   ...                     4 -816.000000  -43 -2131 -3264   
3  0.0   0   0   ...                     2 -532.000000 -382  -682 -1064   
4  0.0   0   0   ...                     3  -54.333333  -11  -121  -163   

  AMT_ANNUITY                                      
        count         mean      max  min      sum  
0           7  3545.357143  10822.5  0.0  24817.5  
1           7     0.000000      0.0  0.0      0.0  
2           0          NaN      NaN  NaN      0.0  
3           0          NaN      NaN  NaN      0.0  
4           3  1420.500000   4261.5  0.0   4261.5  

[5 rows x 61 columns]

In [10]:
columns = ['SK_ID_CURR']

for var in bureau_agg.columns.levels[0]:
    
    if var != 'SK_ID_CURR':
        for stat in bureau_agg.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s'%(var, stat))

In [11]:
bureau_agg.columns = columns
bureau_agg.head()

SK_ID_CURR  bureau_DAYS_CREDIT_count  bureau_DAYS_CREDIT_mean  \
0      100001                         7              -735.000000   
1      100002                         8              -874.000000   
2      100003                         4             -1400.750000   
3      100004                         2              -867.000000   
4      100005                         3              -190.666667   

   bureau_DAYS_CREDIT_max  bureau_DAYS_CREDIT_min  bureau_DAYS_CREDIT_sum  \
0                     -49                   -1572                   -5145   
1                    -103                   -1437                   -6992   
2                    -606                   -2586                   -5603   
3                    -408                   -1326                   -1734   
4                     -62                    -373                    -572   

   bureau_CREDIT_DAY_OVERDUE_count  bureau_CREDIT_DAY_OVERDUE_mean  \
0                                7                             0.0   
1                                8                             0.0   
2                                4                             0.0   
3                                2                             0.0   
4                                3                             0.0   

   bureau_CREDIT_DAY_OVERDUE_max  bureau_CREDIT_DAY_OVERDUE_min  \
0                              0                              0   
1                              0                              0   
2                              0                              0   
3                              0                              0   
4                              0                              0   

            ...            bureau_DAYS_CREDIT_UPDATE_count  \
0           ...                                          7   
1           ...                                          8   
2           ...                                          4   
3           ...                                          2   
4           ...                                          3   

   bureau_DAYS_CREDIT_UPDATE_mean  bureau_DAYS_CREDIT_UPDATE_max  \
0                      -93.142857                             -6   
1                     -499.875000                             -7   
2                     -816.000000                            -43   
3                     -532.000000                           -382   
4                      -54.333333                            -11   

   bureau_DAYS_CREDIT_UPDATE_min  bureau_DAYS_CREDIT_UPDATE_sum  \
0                           -155                           -652   
1                          -1185                          -3999   
2                          -2131                          -3264   
3                           -682                          -1064   
4                           -121                           -163   

   bureau_AMT_ANNUITY_count  bureau_AMT_ANNUITY_mean  bureau_AMT_ANNUITY_max  \
0                         7              3545.357143                 10822.5   
1                         7                 0.000000                     0.0   
2                         0                      NaN                     NaN   
3                         0                      NaN                     NaN   
4                         3              1420.500000                  4261.5   

   bureau_AMT_ANNUITY_min  bureau_AMT_ANNUITY_sum  
0                     0.0                 24817.5  
1                     0.0                     0.0  
2                     NaN                     0.0  
3                     NaN                     0.0  
4                     0.0                  4261.5  

[5 rows x 61 columns]

In [12]:
train = train.merge(bureau_agg, on='SK_ID_CURR', how='left')
train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

            ...            bureau_DAYS_CREDIT_UPDATE_count  \
0           ...                                        8.0   
1           ...                                        4.0   
2           ...                                        2.0   
3           ...                                        NaN   
4           ...                                        1.0   

  bureau_DAYS_CREDIT_UPDATE_mean bureau_DAYS_CREDIT_UPDATE_max  \
0                       -499.875                          -7.0   
1                       -816.000                         -43.0   
2                       -532.000                        -382.0   
3                            NaN                           NaN   
4                       -783.000                        -783.0   

  bureau_DAYS_CREDIT_UPDATE_min bureau_DAYS_CREDIT_UPDATE_sum  \
0                       -1185.0                       -3999.0   
1                       -2131.0                       -3264.0   
2                        -682.0                       -1064.0   
3                           NaN                           NaN   
4                        -783.0                        -783.0   

  bureau_AMT_ANNUITY_count  bureau_AMT_ANNUITY_mean  bureau_AMT_ANNUITY_max  \
0                      7.0                      0.0                     0.0   
1                      0.0                      NaN                     NaN   
2                      0.0                      NaN                     NaN   
3                      NaN                      NaN                     NaN   
4                      0.0                      NaN                     NaN   

   bureau_AMT_ANNUITY_min  bureau_AMT_ANNUITY_sum  
0                     0.0                     0.0  
1                     NaN                     0.0  
2                     NaN                     0.0  
3                     NaN                     NaN  
4                     NaN                     0.0  

[5 rows x 183 columns]

In [13]:
new_corrs = []

for col in columns:
    corr = train['TARGET'].corr(train[col])
    new_corrs.append((col, corr))

In [14]:
new_corrs = sorted(new_corrs, key=lambda x: abs(x[1]), reverse=True)

In [15]:
new_corrs[:15]

[('bureau_DAYS_CREDIT_mean', 0.089728967219981137),
 ('bureau_DAYS_CREDIT_min', 0.075248251030103602),
 ('bureau_DAYS_CREDIT_UPDATE_mean', 0.068927352669686734),
 ('bureau_DAYS_ENDDATE_FACT_min', 0.055887379843920768),
 ('bureau_DAYS_CREDIT_ENDDATE_sum', 0.053734895601020502),
 ('bureau_DAYS_ENDDATE_FACT_mean', 0.053199625857586162),
 ('bureau_DAYS_CREDIT_max', 0.04978205463997299),
 ('bureau_DAYS_ENDDATE_FACT_sum', 0.048853502611115894),
 ('bureau_DAYS_CREDIT_ENDDATE_mean', 0.046982754334835494),
 ('bureau_DAYS_CREDIT_UPDATE_min', 0.042863922470730155),
 ('bureau_DAYS_CREDIT_sum', 0.041999824814846716),
 ('bureau_DAYS_CREDIT_UPDATE_sum', 0.04140363535306002),
 ('bureau_DAYS_CREDIT_ENDDATE_max', 0.036589634696329094),
 ('bureau_DAYS_CREDIT_ENDDATE_min', 0.034281109921616024),
 ('bureau_DAYS_ENDDATE_FACT_count', -0.030492306653325495)]

In [16]:
kde_target(train, 'bureau_DAYS_CREDIT_mean')

The correlation between bureau_DAYS_CREDIT_mean and TARGET is 0.08972896722
Median value for loan that was not repaid is -835.333333333
Median value for loan that was repaid is -1067.0


In [62]:
def agg_numeric(df, df_name, group_var):
    
    
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns=col)
            
    grouping_column = df[group_var]
    numeric_df = df.select_dtypes('number')
   
    
    numeric_df[group_var] = grouping_column #create the grouping column
    
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
    
    columns = [group_var]
    
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s'%(df_name, var, stat))
   
    agg.columns = columns
    return agg

In [65]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')

In [67]:
bureau_agg_new.head()

SK_ID_CURR  bureau_DAYS_CREDIT_count  bureau_DAYS_CREDIT_mean  \
0      100001                         7              -735.000000   
1      100002                         8              -874.000000   
2      100003                         4             -1400.750000   
3      100004                         2              -867.000000   
4      100005                         3              -190.666667   

   bureau_DAYS_CREDIT_max  bureau_DAYS_CREDIT_min  bureau_DAYS_CREDIT_sum  \
0                     -49                   -1572                   -5145   
1                    -103                   -1437                   -6992   
2                    -606                   -2586                   -5603   
3                    -408                   -1326                   -1734   
4                     -62                    -373                    -572   

   bureau_CREDIT_DAY_OVERDUE_count  bureau_CREDIT_DAY_OVERDUE_mean  \
0                                7                             0.0   
1                                8                             0.0   
2                                4                             0.0   
3                                2                             0.0   
4                                3                             0.0   

   bureau_CREDIT_DAY_OVERDUE_max  bureau_CREDIT_DAY_OVERDUE_min  \
0                              0                              0   
1                              0                              0   
2                              0                              0   
3                              0                              0   
4                              0                              0   

            ...            bureau_DAYS_CREDIT_UPDATE_count  \
0           ...                                          7   
1           ...                                          8   
2           ...                                          4   
3           ...                                          2   
4           ...                                          3   

   bureau_DAYS_CREDIT_UPDATE_mean  bureau_DAYS_CREDIT_UPDATE_max  \
0                      -93.142857                             -6   
1                     -499.875000                             -7   
2                     -816.000000                            -43   
3                     -532.000000                           -382   
4                      -54.333333                            -11   

   bureau_DAYS_CREDIT_UPDATE_min  bureau_DAYS_CREDIT_UPDATE_sum  \
0                           -155                           -652   
1                          -1185                          -3999   
2                          -2131                          -3264   
3                           -682                          -1064   
4                           -121                           -163   

   bureau_AMT_ANNUITY_count  bureau_AMT_ANNUITY_mean  bureau_AMT_ANNUITY_max  \
0                         7              3545.357143                 10822.5   
1                         7                 0.000000                     0.0   
2                         0                      NaN                     NaN   
3                         0                      NaN                     NaN   
4                         3              1420.500000                  4261.5   

   bureau_AMT_ANNUITY_min  bureau_AMT_ANNUITY_sum  
0                     0.0                 24817.5  
1                     0.0                     0.0  
2                     NaN                     0.0  
3                     NaN                     0.0  
4                     0.0                  4261.5  

[5 rows x 61 columns]

In [68]:
bureau_agg.head()

SK_ID_CURR  bureau_DAYS_CREDIT_count  bureau_DAYS_CREDIT_mean  \
0      100001                         7              -735.000000   
1      100002                         8              -874.000000   
2      100003                         4             -1400.750000   
3      100004                         2              -867.000000   
4      100005                         3              -190.666667   

   bureau_DAYS_CREDIT_max  bureau_DAYS_CREDIT_min  bureau_DAYS_CREDIT_sum  \
0                     -49                   -1572                   -5145   
1                    -103                   -1437                   -6992   
2                    -606                   -2586                   -5603   
3                    -408                   -1326                   -1734   
4                     -62                    -373                    -572   

   bureau_CREDIT_DAY_OVERDUE_count  bureau_CREDIT_DAY_OVERDUE_mean  \
0                                7                             0.0   
1                                8                             0.0   
2                                4                             0.0   
3                                2                             0.0   
4                                3                             0.0   

   bureau_CREDIT_DAY_OVERDUE_max  bureau_CREDIT_DAY_OVERDUE_min  \
0                              0                              0   
1                              0                              0   
2                              0                              0   
3                              0                              0   
4                              0                              0   

            ...            bureau_DAYS_CREDIT_UPDATE_count  \
0           ...                                          7   
1           ...                                          8   
2           ...                                          4   
3           ...                                          2   
4           ...                                          3   

   bureau_DAYS_CREDIT_UPDATE_mean  bureau_DAYS_CREDIT_UPDATE_max  \
0                      -93.142857                             -6   
1                     -499.875000                             -7   
2                     -816.000000                            -43   
3                     -532.000000                           -382   
4                      -54.333333                            -11   

   bureau_DAYS_CREDIT_UPDATE_min  bureau_DAYS_CREDIT_UPDATE_sum  \
0                           -155                           -652   
1                          -1185                          -3999   
2                          -2131                          -3264   
3                           -682                          -1064   
4                           -121                           -163   

   bureau_AMT_ANNUITY_count  bureau_AMT_ANNUITY_mean  bureau_AMT_ANNUITY_max  \
0                         7              3545.357143                 10822.5   
1                         7                 0.000000                     0.0   
2                         0                      NaN                     NaN   
3                         0                      NaN                     NaN   
4                         3              1420.500000                  4261.5   

   bureau_AMT_ANNUITY_min  bureau_AMT_ANNUITY_sum  
0                     0.0                 24817.5  
1                     0.0                     0.0  
2                     NaN                     0.0  
3                     NaN                     0.0  
4                     0.0                  4261.5  

[5 rows x 61 columns]

In [69]:
def target_cors(df):
    
    corrs = []
    
    for col in df.columns:
        print(col)
        if col != 'TARGET':
            corr = df['TARGET'].corr(df[col])
            corrs.append((col,corr))
    
    corrs = sorted(corrs, key= lambda x: abs(x[1]), reverse=True)
    return corrs

In [ ]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']

In [ ]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
categorical_grouped.head()

In [75]:
group_var = 'SK_ID_CURR'

columns = []

for var in categorical_grouped.columns.levels[0]:
    
    if var != group_var:
        
        for stat in ['count', 'count_norm']:
            columns.append('%s_%s'%(var, stat))
            
categorical_grouped.columns = columns
categorical_grouped.head()

CREDIT_ACTIVE_Active_count  CREDIT_ACTIVE_Active_count_norm  \
SK_ID_CURR                                                                
100001                               3                         0.428571   
100002                               2                         0.250000   
100003                               1                         0.250000   
100004                               0                         0.000000   
100005                               2                         0.666667   

            CREDIT_ACTIVE_Bad debt_count  CREDIT_ACTIVE_Bad debt_count_norm  \
SK_ID_CURR                                                                    
100001                                 0                                0.0   
100002                                 0                                0.0   
100003                                 0                                0.0   
100004                                 0                                0.0   
100005                                 0                                0.0   

            CREDIT_ACTIVE_Closed_count  CREDIT_ACTIVE_Closed_count_norm  \
SK_ID_CURR                                                                
100001                               4                         0.571429   
100002                               6                         0.750000   
100003                               3                         0.750000   
100004                               2                         1.000000   
100005                               1                         0.333333   

            CREDIT_ACTIVE_Sold_count  CREDIT_ACTIVE_Sold_count_norm  \
SK_ID_CURR                                                            
100001                             0                            0.0   
100002                             0                            0.0   
100003                             0                            0.0   
100004                             0                            0.0   
100005                             0                            0.0   

            CREDIT_CURRENCY_currency 1_count  \
SK_ID_CURR                                     
100001                                     7   
100002                                     8   
100003                                     4   
100004                                     2   
100005                                     3   

            CREDIT_CURRENCY_currency 1_count_norm  \
SK_ID_CURR                                          
100001                                        1.0   
100002                                        1.0   
100003                                        1.0   
100004                                        1.0   
100005                                        1.0   

                               ...                       \
SK_ID_CURR                     ...                        
100001                         ...                        
100002                         ...                        
100003                         ...                        
100004                         ...                        
100005                         ...                        

            CREDIT_TYPE_Microloan_count  CREDIT_TYPE_Microloan_count_norm  \
SK_ID_CURR                                                                  
100001                                0                               0.0   
100002                                0                               0.0   
100003                                0                               0.0   
100004                                0                               0.0   
100005                                0                               0.0   

            CREDIT_TYPE_Mobile operator loan_count  \
SK_ID_CURR                                           
100001                                           0   
100002                                           0   
100003                       

In [76]:
train = train.merge(categorical_grouped, on='SK_ID_CURR', how='left')

In [83]:
def count_categorical(df, group_var, df_name):
    columns = []
    
    categorical = pd.get_dummies(df.select_dtypes('object'))
    categorical[group_var] = df[group_var]
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    for var in categorical.columns.levels[0]:
        if var != group_var:
            for stat in ['count', 'count_norm']:
                columns.append('%s_%s_%s'%(df_name, var, stat))
    categorical.columns = columns
    return categorical

In [84]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')

In [86]:
bureau_balance = pd.read_csv('bureau_balance.csv')

In [87]:
bureau_balance_counts = count_categorical(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau_balance')

In [ ]:
bureau_balance_counts.head()

In [ ]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau_balance')
bureau_balance_agg.head()

In [100]:
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index=True, left_on='SK_ID_BUREAU', how='outer')

In [101]:
bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on='SK_ID_BUREAU', how='left')
bureau_by_loan.head()

SK_ID_BUREAU  bureau_balance_MONTHS_BALANCE_count  \
0       5001709                                   97   
1       5001710                                   83   
2       5001711                                    4   
3       5001712                                   19   
4       5001713                                   22   

   bureau_balance_MONTHS_BALANCE_mean  bureau_balance_MONTHS_BALANCE_max  \
0                               -48.0                                  0   
1                               -41.0                                  0   
2                                -1.5                                  0   
3                                -9.0                                  0   
4                               -10.5                                  0   

   bureau_balance_MONTHS_BALANCE_min  bureau_balance_MONTHS_BALANCE_sum  \
0                                -96                              -4656   
1                                -82                              -3403   
2                                 -3                                 -6   
3                                -18                               -171   
4                                -21                               -231   

   bureau_balance_STATUS_0_count  bureau_balance_STATUS_0_count_norm  \
0                              0                            0.000000   
1                              5                            0.060241   
2                              3                            0.750000   
3                             10                            0.526316   
4                              0                            0.000000   

   bureau_balance_STATUS_1_count  bureau_balance_STATUS_1_count_norm  \
0                              0                                 0.0   
1                              0                                 0.0   
2                              0                                 0.0   
3                              0                                 0.0   
4                              0                                 0.0   

      ...      bureau_balance_STATUS_3_count_norm  \
0     ...                                     0.0   
1     ...                                     0.0   
2     ...                                     0.0   
3     ...                                     0.0   
4     ...                                     0.0   

   bureau_balance_STATUS_4_count  bureau_balance_STATUS_4_count_norm  \
0                              0                                 0.0   
1                              0                                 0.0   
2                              0                                 0.0   
3                              0                                 0.0   
4                              0                                 0.0   

   bureau_balance_STATUS_5_count  bureau_balance_STATUS_5_count_norm  \
0                              0                                 0.0   
1                              0                                 0.0   
2                              0                                 0.0   
3                              0                                 0.0   
4                              0                                 0.0   

   bureau_balance_STATUS_C_count  bureau_balance_STATUS_C_count_norm  \
0                             86                            0.886598   
1                             48                            0.578313   
2                              0                            0.000000   
3                              9                            0.473684   
4                              0                            0.000000   

   bureau_balance_STATUS_X_count  bureau_balance_STATUS_X_count_norm  \
0                             11                            0.113402   
1                             30                            0.361446   
2                              1                          

In [103]:
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']), df_name='client', group_var='SK_ID_CURR')
bureau_balance_by_client.head()

SK_ID_CURR  client_bureau_balance_MONTHS_BALANCE_count_count  \
0    100001.0                                                 7   
1    100002.0                                                 8   
2    100005.0                                                 3   
3    100010.0                                                 2   
4    100013.0                                                 4   

   client_bureau_balance_MONTHS_BALANCE_count_mean  \
0                                        24.571429   
1                                        13.750000   
2                                         7.000000   
3                                        36.000000   
4                                        57.500000   

   client_bureau_balance_MONTHS_BALANCE_count_max  \
0                                              52   
1                                              22   
2                                              13   
3                                              36   
4                                              69   

   client_bureau_balance_MONTHS_BALANCE_count_min  \
0                                               2   
1                                               4   
2                                               3   
3                                              36   
4                                              40   

   client_bureau_balance_MONTHS_BALANCE_count_sum  \
0                                             172   
1                                             110   
2                                              21   
3                                              72   
4                                             230   

   client_bureau_balance_MONTHS_BALANCE_mean_count  \
0                                                7   
1                                                8   
2                                                3   
3                                                2   
4                                                4   

   client_bureau_balance_MONTHS_BALANCE_mean_mean  \
0                                      -11.785714   
1                                      -21.875000   
2                                       -3.000000   
3                                      -46.000000   
4                                      -28.250000   

   client_bureau_balance_MONTHS_BALANCE_mean_max  \
0                                           -0.5   
1                                           -1.5   
2                                           -1.0   
3                                          -19.5   
4                                          -19.5   

   client_bureau_balance_MONTHS_BALANCE_mean_min  \
0                                          -25.5   
1                                          -39.5   
2                                           -6.0   
3                                          -72.5   
4                                          -34.0   

                       ...                        \
0                      ...                         
1                      ...                         
2                      ...                         
3                      ...                         
4                      ...                         

   client_bureau_balance_STATUS_X_count_count  \
0                                           7   
1                                           8   
2                                           3   
3                                           2   
4                                           4   

   client_bureau_balance_STATUS_X_count_mean  \
0                                   4.285714   
1                                   1.875000   
2                                   0.666667   
3                                   0.000000   
4                                  10.250000   

   client_bureau_balance_STATUS_X_count_max  \
0                                         9   
1                                         3   
2         